In [56]:
import requests
import re

In [57]:
user_url = "https://api.github.com/users/ptrgags"

In [58]:
user_info = requests.get(user_url).json()
repos_url = user_info[u'repos_url']

In [66]:
def process_repo(repo):
    info = {}
    info['title'] = repo['name']
    
    #Description and years
    description_splitter = " (?=\()" #matches the space before the date
    description_parts = re.split(description_splitter, repo['description'])
    info['description']  = description_parts[0]
    try:
        info['years'] = description_parts[1][1:-1]
    except IndexError:
        info['years'] = ''
    
    #Tags
    #TODO: Prerelease/Release status
    tags_url = repo['tags_url']
    tags_info = requests.get(tags_url).json()
    info['tags'] = [tag['name'] for tag in tags_info]
    
    #Milestones
    #TODO: how to sort these?
    milestones_url = repo['milestones_url'].replace("{/number}","")
    milestones_info = requests.get(milestones_url).json()
    info['milestones'] = []
    for milestone in milestones_info:
        milestone_data = {}
        milestone_data['name'] = milestone['title']
        milestone_data['open'] = milestone['open_issues']
        milestone_data['closed'] = milestone['closed_issues']
        milestone_data['status'] = milestone['state']
        milestone_data['update_time'] = milestone['updated_at']
        info['milestones'].append(milestone_data)
    
    #Issues
    
    #Commits
    #TODO: put together a tree from branches and commits
    commits_url = repo['commits_url'].replace("{/sha}", "")
    commits_info = requests.get(commits_url).json()
    info['commits'] = []
    for commit in commits_info:
        commit_info = {}
        commit_info['sha'] = commit['sha']
        commit_info['parents'] = [p['sha'] for p in commit['parents']]
        info['commits'].append(commit_info)
    
    
    #Branches
    branches_url = repo['branches_url'].replace("{/branch}", "")
    branches_info = requests.get(branches_url).json()
    info['branches'] = [(b['name'], b['commit']['sha']) for b in branches_info]
    
    #Contributors
    
    #Links
    info['github_link'] = repo['html_url']
    info['website'] = repo['homepage'] or ''
        
    
    return info

In [67]:
repos_info = requests.get(repos_url).json()
process_repo(repos_info[9])

{'branches': [(u'content', u'f240ce8c7b9386bd16e8adbee5fc0322ee93a51d'),
  (u'develop', u'31398e5f9435e242e7863d03e3ab7c42bba5faa0'),
  (u'json', u'10f09f3ec93f5bdf769eceabf617c89a0dc18733'),
  (u'master', u'0827bc41ef5840b6309aeeaf44eec7b5ba0a437a'),
  (u'research', u'a687cb938047de1619a18d6406594d7470ab9889')],
 'commits': [{'parents': [u'10d94b9e47befb2f72cd4f96661ed3042a24f750'],
   'sha': u'0827bc41ef5840b6309aeeaf44eec7b5ba0a437a'},
  {'parents': [u'404dccf92b9de7a8f61d9870ec97fc01290a8229',
    u'10f09f3ec93f5bdf769eceabf617c89a0dc18733'],
   'sha': u'10d94b9e47befb2f72cd4f96661ed3042a24f750'},
  {'parents': [u'288def75b21e023ff9dde640c064dd9d7ade2cdd'],
   'sha': u'10f09f3ec93f5bdf769eceabf617c89a0dc18733'},
  {'parents': [u'27a609ef1ffdd44f1f96134dde3f57c9cb36f838'],
   'sha': u'288def75b21e023ff9dde640c064dd9d7ade2cdd'},
  {'parents': [u'15f043b5400d995428fdf81ac65f31bcefae6aed'],
   'sha': u'27a609ef1ffdd44f1f96134dde3f57c9cb36f838'},
  {'parents': [u'a7d9a06726aeba1a5811d38